In [3]:
import pandas as pd
import os


df = pd.read_csv("DKSalaries.csv")
df = df[["Roster Position", "Name + ID", "Salary", "TeamAbbrev"]]


current_directory = os.getcwd()

# Construct the path to the file in the higher directory
higher_directory = os.path.abspath(
    os.path.join(current_directory, os.pardir, os.pardir)
)
file_in_higher_directory_path = os.path.join(higher_directory, "players.csv")
players = pd.read_csv(file_in_higher_directory_path)

In [4]:
players.head(5)

,player_id,full_name,position
0,1630173,Precious Achiuwa,Forward
1,203500,Steven Adams,Center
2,1628389,Bam Adebayo,Center-Forward
3,1630534,Ochai Agbaji,Guard
4,1630583,Santi Aldama,Forward-Center


In [5]:
def appFunc_getID(nameAndID):
    draftking_full_name = nameAndID.split("(")[0].strip().lower()
    filtered_df = players[players["full_name"].str.lower() == draftking_full_name]

    if filtered_df.empty:
        print("No row found for", draftking_full_name)
        return None

    return int(filtered_df.iloc[0]["player_id"])


df["player_id"] = df["Name + ID"].apply(appFunc_getID)
df.head(5)

,Roster Position,Name + ID,Salary,TeamAbbrev,player_id
0,PF/C/F/UTIL,Giannis Antetokounmpo (32200475),11700,MIL,203507
1,C/UTIL,Joel Embiid (32200486),11500,PHI,203954
2,PG/G/UTIL,Damian Lillard (32200421),8800,MIL,203081
3,C/UTIL,Alperen Sengun (32200611),8600,HOU,1630578
4,PG/G/UTIL,Tyrese Maxey (32200576),8300,PHI,1630178


In [6]:
salary_dict = {}
team_dict = {}

for _, row in df.iterrows():
    salary_dict[row["player_id"]] = row["Salary"]
    team_dict[row["player_id"]] = row["TeamAbbrev"]

In [ ]:
# 1. Make dict of roster positions with players as array
# 2. Generate N random valid lineups
# 3. Simulate, get overall rankings. Keep top 5-10% just as they are (elitism). Use top 25% as sure parents, with rest having 33%?
# 4. Inherit by:
# for each survivor, find M other random survivors. Make X children by randomly deciding traits from parents
# 5. Mutate children by:
# for each child, randomly swap out T players, with T changing over time?
# 6. Add Q purely random lineups
# 6. Save best, repeat 2-6 until satisfied

### 1. Make dict of roster positions with players as array

In [7]:
player_dict = {}
player_dict["PG"] = []
player_dict["SG"] = []
player_dict["SF"] = []
player_dict["PF"] = []
player_dict["C"] = []
player_dict["G"] = []
player_dict["F"] = []
player_dict["UTIL"] = []

for _, row in df.iterrows():
    if "PG" in row["Roster Position"]:
        player_dict["PG"].append(row["player_id"])
    if "SG" in row["Roster Position"]:
        player_dict["SG"].append(row["player_id"])
    if "SF" in row["Roster Position"]:
        player_dict["SF"].append(row["player_id"])
    if "PF" in row["Roster Position"]:
        player_dict["PF"].append(row["player_id"])
    if "C" in row["Roster Position"]:
        player_dict["C"].append(row["player_id"])
    if "G" in row["Roster Position"]:
        player_dict["G"].append(row["player_id"])
    if "F" in row["Roster Position"]:
        player_dict["F"].append(row["player_id"])
    if "UTIL" in row["Roster Position"]:
        player_dict["UTIL"].append(row["player_id"])

len(player_dict["UTIL"])

72

In [8]:
from operator import itemgetter


def lineup_is_valid(lineup_array):
    """Assumes that the array of id's has one per correct slot."""
    if len(lineup_array) != 8:  # check is right length
        return False
    if len(set(lineup_array)) != len(lineup_array):  # check for dupes
        return False
    salaries = itemgetter(*lineup_array)(salary_dict)
    if sum(salaries) > 50000:  # check in budget
        return False
    teams = itemgetter(*lineup_array)(team_dict)
    if len(set(teams)) < 2:  # Check if has both teams
        return False
    return True

### 2. Generate N random valid lineups

In [14]:
import math
import random


def generate_n_pure_random_lineups(num_lineups):
    lineups = []
    max_length = num_lineups
    print(max_length)
    while len(lineups) < max_length:
        pg = random.choice(player_dict["PG"])
        sg = random.choice(player_dict["SG"])
        sf = random.choice(player_dict["SF"])
        pf = random.choice(player_dict["PF"])
        c = random.choice(player_dict["C"])
        g = random.choice(player_dict["G"])
        f = random.choice(player_dict["F"])
        util = random.choice(player_dict["UTIL"])
        new_lineup = [pg, sg, sf, pf, c,g,f,util]
        if lineup_is_valid(new_lineup):
            lineups.append(new_lineup)
    return lineups

generate_n_pure_random_lineups(1)

1


[[203506, 1629001, 1628415, 1626171, 203954, 1631260, 1641748, 1641753]]

### 3. Simulate Trials and get Parents/Elites

In [10]:
import json
import os


json_file_path = os.path.join(
    higher_directory, "models", "simpledfs.json"
)

with open(json_file_path, "r") as json_file:
    # Load the JSON data into a dictionary
    trials = json.load(json_file)

In [ ]:
import numpy as np


def get_rankings(lineup_array, trials, num_trials):

    # 0. Make an array of length lineup_array to store rankings
    rankings_list = [0] * len(lineup_array)

    # 1. Pick a random draw num (1-2000) FOR EACH PLAYER

    for i in range(num_trials):
        individual_player_scores = {}
        lineup_scores = {}
        # 2. Make a dict of player:score for that trial
        for player_id in trials.keys():
            individual_player_scores[player_id] = random.choice(trials[player_id])
        # 3. For each lineup, for each player in that lineup add that score to the index in the array
        for j in range(len(lineup_array)):
            curr_lineup = lineup_array[j]
            lineup_score = 0
            for player in curr_lineup:
                lineup_score += individual_player_scores[player]
            lineup_scores[j] = lineup_score

        # 4. Add rankings to rank array
        sorted_scores_list = sorted(
            lineup_scores.items(), key=lambda item: item[1], reverse=True
        )
        counter = 1
        for sorted_score_tuple in sorted_scores_list:
            rankings_list[sorted_score_tuple[0]] += counter
            counter += 1

    # 5. Return rankings array

    return rankings_list



def get_parents(
    lineup_array, ranking_list, sure_parent_percent, possible_parent_percent
):

    # Get sure parent percentile of rankings
    threshold_value = np.percentile(ranking_list, sure_parent_percent)

    # Find indices of values above the threshold
    indices_above_threshold = np.where(ranking_list > threshold_value)[0]

    # get indices of all rankings below (?) that percentile, add to array
    indices_below_threshold = np.where(ranking_list <= threshold_value)[0]

    # for indices not in that list, if random num > possible_parent_percent, add to array
    for index in indices_below_threshold:
        if random.random() < possible_parent_percent:
            indices_above_threshold.append(index)

    # return array
    selected_lineups = lineup_array[indices_above_threshold]
    return selected_lineups

### 4. Inherit

In [ ]:
def make_children(survivor_lineups, num_children):
    children_lineups = []

    # while len(children) < num_children
    while len(children_lineups) < num_children:
        # pick two random survivors
        first_parent = random.choice(survivor_lineups)
        second_parent = random.choice(survivor_lineups)
        # for each position, randomly pick between the two parents
        child_lineup = [0] * len(first_parent)
        for player_position in range(len(first_parent)):
            child_lineup[player_position] = random.choice(
                [first_parent[player_position], second_parent[player_position]]
            )
    # if not a valid thing, continue
        if lineup_is_valid(child_lineup):
            children_lineups.append(child_lineup)
            
    return children_lineups

### 5. Mutate

In [ ]:
def mutate_children(children_lineups, num_mutations, percent_mutated):
    # sample percent_mutated children
    num_elements_to_sample = int((percent_mutated / 100) * len(children_lineups))
    random_child_lineups = np.random.choice(
        children_lineups, size=num_elements_to_sample, replace=False
    )
    out_children = np.setdiff1d(children_lineups, random_child_lineups)
    # for each, add num_mutations. If not valid, try another set of mutations until it does.
    for child in random_child_lineups:
        temp_child = child
        for i in range(len(num_mutations)):
            position_to_mutate = random.randint(0, len(child) - 1)
            if position_to_mutate == 0:
                temp_child[position_to_mutate] = random.choice(player_dict["PG"])
            if position_to_mutate == 1:
                temp_child[position_to_mutate] = random.choice(player_dict["SG"])
            if position_to_mutate == 2:
                temp_child[position_to_mutate] = random.choice(player_dict["SF"])
            if position_to_mutate == 3:
                temp_child[position_to_mutate] = random.choice(player_dict["PF"])
            if position_to_mutate == 4:
                temp_child[position_to_mutate] = random.choice(player_dict["C"])
            if position_to_mutate == 5:
                temp_child[position_to_mutate] = random.choice(player_dict["G"])
            if position_to_mutate == 6:
                temp_child[position_to_mutate] = random.choice(player_dict["F"])
            if position_to_mutate == 7:
                temp_child[position_to_mutate] = random.choice(player_dict["UTIL"])

            while not lineup_is_valid(temp_child):
                temp_child = child
                if position_to_mutate == 0:
                    temp_child[position_to_mutate] = random.choice(player_dict["PG"])
                if position_to_mutate == 1:
                    temp_child[position_to_mutate] = random.choice(player_dict["SG"])
                if position_to_mutate == 2:
                    temp_child[position_to_mutate] = random.choice(player_dict["SF"])
                if position_to_mutate == 3:
                    temp_child[position_to_mutate] = random.choice(player_dict["PF"])
                if position_to_mutate == 4:
                    temp_child[position_to_mutate] = random.choice(player_dict["C"])
                if position_to_mutate == 5:
                    temp_child[position_to_mutate] = random.choice(player_dict["G"])
                if position_to_mutate == 6:
                    temp_child[position_to_mutate] = random.choice(player_dict["F"])
                if position_to_mutate == 7:
                    temp_child[position_to_mutate] = random.choice(player_dict["UTIL"])

        out_children.append(temp_child)
    # return mutated children
    return out_children

## Evolution

In [ ]:
best_lineups = {}


def run_ga(
    num_gens,
    num_trials,
    sure_parent_percent,
    possible_parent_percent,
    num_mutations,
    percent_mutated,
):
    # random start
    population = generate_n_pure_random_lineups(100)
    GOAT_lineups = []
    # for num_gens
    for i in range(1, num_gens + 1):
        # run get rankings on pop
        rankings_by_index = get_rankings(population, trials, num_trials)
        # save best 2? in best_lineups[gen_num] = [[...],[...]], or csv
        indices_of_highest_values = np.argsort(rankings_by_index)[-2:]
        best_lineups[i] = {
            rankings_by_index[indices_of_highest_values[0]]: population[
                indices_of_highest_values[0]
            ],
            rankings_by_index[indices_of_highest_values[1]]: population[
                indices_of_highest_values[1]
            ],
        }
        GOAT_lineups.append(population[indices_of_highest_values[0]])
        GOAT_lineups.append(population[indices_of_highest_values[1]])
        # print best ranking
        print(
            "Best score of gen", i, "is", rankings_by_index[indices_of_highest_values]
        )
        # run make children
        parents = get_parents(
            population, rankings_by_index, sure_parent_percent, possible_parent_percent
        )
        num_children = int((1 - (1 / i)) * 100)
        num_random = 100 - num_children
        children = make_children(parents, num_children)

        # run mutate children
        mutated_children = mutate_children(children, num_mutations, percent_mutated)

        # add random for the rest
        new_random_kids = generate_n_pure_random_lineups(num_random)

        # reset population
        population = mutate_children + new_random_kids
    # return best of the best

    GOAT_rankings = get_rankings(GOAT_lineups, trials, num_trials)
    GOAT_score_pairs = list(zip(GOAT_lineups, GOAT_rankings))
    sorted_pairs = sorted(GOAT_score_pairs, key=lambda pair: pair[1], reverse=True)

    # Print player IDs in order of scores in reverse order
    for lineup, score in sorted_pairs:
        print(f"Lineup: {lineup}, Score: {score}")